##### Main.py

The FastAPI entry point - this is the file you run to start the server. It's responsible for configuring the application, registering middleware and routes, and executing startup tasks

Import stuff and define constants

BASE_DIR is the absolute path to the direectory containing main.py itself (backend/). All other paths are relative to it. 'Using os.path.abspath(__file_)' insead of a hard-coded string ensures that paths resolve correctly even if the server is launched from a differently named directory.

In [ ]:
import os

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

from core.ingestion.registry_loader import load_registry
from core.state_store.database import init_db
from api.routes import router

# Constants
BASE_DIR     = os.path.dirname(os.path.abspath(__file__))
DATA_DIR     = os.path.join(BASE_DIR, "..", "data")
DB_PATH      = os.path.join(DATA_DIR, "asHDT.db")
ARCHIVE_ROOT = os.path.join(DATA_DIR, "archive")
REPORTS_ROOT = os.path.join(DATA_DIR, "reports")
REGISTRY_PATH = os.path.join(BASE_DIR, "..", "registry", "module_registry.json")

**App instantiation and CORS:**

THe FastAPI apllication instance is created. CORS (cross-origin resource sharing) middleware is added, which just makes sure that all the different moving parts konw which ports to communicate with (Svelte frontend runs on port 5173 but the backend on port 8000, so this function tells the browser that requests from 5173 are also fine).

The router defined in routes.py is also registered, making the three endpoints /registry, /subjects, and /timegraph active.

In [ ]:
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:5173"],
    allow_methods=["*"],
    allow_headers=["*"],
)

app.include_router(router)


Startup handler

This function runs just once, on startup. It creates the SQLite database file and tables if they don't already exist, it loads in the module registry and stores in in app.state, as well as other path strings that it will need down the road, making them way easier to access than importing them from main.py every time they're needed.

In [ ]:
@app.on_event("startup")
def startup():
    init_db(DB_PATH)

    app.state.registry     = load_registry(REGISTRY_PATH)
    app.state.db_path      = DB_PATH
    app.state.archive_root = ARCHIVE_ROOT
    app.state.reports_root = REPORTS_ROOT